# Продолжаем про декораторы 

## Замена исходной функции

Функция, возвращаемая декоратором, заменяет декорируемую

In [136]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [139]:
def target():
    print('running target()')

In [140]:
@deco
def target():
    print('running target()')

In [141]:
target()

running inner()


In [143]:
target  # now reference to inner 

<function __main__.deco.<locals>.inner()>

## Running at import time

Key feature декораторов в том, что они применяются сразу после объявления декорируемой функции во время загрузки модуля в Python 

In [147]:
registry = []

def register(func):
    print(f'running register {func}')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
def run():
    print('running run()')
    print('registry', registry)
    f1()
    f2()
    f3()

running register <function f1 at 0x7f6c72626f80>
running register <function f2 at 0x7f6c72626ef0>


In [148]:
run()

running run()
registry [<function f1 at 0x7f6c72626f80>, <function f2 at 0x7f6c72626ef0>]
running f1()
running f2()
running f3()


## Декоратор-логгер

### logging

 Модуль `logging` содержит набор функций для логирования различного поведения в вашей программе. Например, для понимания, что ваша программа работает как ожидается, или там идет что-то не так

Есть разные уровни логирования: debug, info, warning, error, critical

In [1]:
import logging

In [2]:
logging.info('Working as expected')
logging.warning('Smth might go wrong, check this!')

Можно логировать в файл (надо начать новую сессию в Python)

In [15]:
import os
import logging

log_filename = os.path.join('data', 'example.log')
log_level = logging.DEBUG

logging.basicConfig(
    filename=log_filename,
    encoding='utf-8',
    level=log_level,
    format='%(asctime)s %(message)s'  # add time
)

logging.debug('One very thorough message to save in our log')
logging.info('Just some info, also in log')
logging.warning('Something might be wrong')
logging.error('Logging our failure')

In [2]:
! cat data/example.log

2022-09-29 03:04:13,025 One very thorough message to save in our log
2022-09-29 03:04:13,025 Just some info, also in log
2022-09-29 03:04:13,025 Something might be wrong
2022-09-29 03:04:13,025 Logging our failure


### Напишем log-декоратор

In [103]:
import logging
import sys

from functools import wraps, partial

In [155]:
logger = logging.getLogger(name='log')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [159]:
def logged(func=None,
           *,
           log_level=logging.INFO,
           log_name=None,
           log_message=None):
    
    if func is None:
        return partial(logged, log_level=log_level, log_name=log_name, log_message=log_message)
    
    log_name = log_name if log_name else func.__name__
    log_message = log_message if log_message else func.__name__
        
    logger = logging.getLogger(log_name)
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        logger.log(log_level, log_message)
        return func(*args, **kwargs)
    
    return wrapper

In [160]:
@logged(log_name='log')
def add(x, y):
    return x + y

In [161]:
add(1, 2)

add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add


INFO:log:add


3

Протестируем с декоратором с прошлых семинаров

In [164]:
def clock(func):
    @wraps(func)
    def clocked(*args):
        t_start = time.time()
        res = func(*args)
        total_time = time.time() - t_start
        print(f'{func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
        return res
    return clocked

In [168]:
@logged(log_name='log')
@clock
def add(x, y):
    time.sleep(2)
    return x + y

In [169]:
add(10, 10**10)

add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add


INFO:log:add


add((10, 10000000000)) -> 10000000010 executed in 2.00s


10000000010

## Декоратор -- assert типов (3 балла)

Мы уже говорили про type hints, а также про то, что никаких ошибок без использования сторонних модулей некорректные типы нам не дадут. Давайте реализуем логику проверки типов с помощью декоратора.

Типы могут быть заданы как стандартными названиями, так и с использованыем модуля typing. Для Any пропускайте проверку

In [11]:
from typing import Any

In [9]:
@check_types
def func(a: int, b: str) -> str:
    return a * b

Стоит выбросить ValueError, если есть несовпадение между реальным и ожидаемым типом у какой-то из переданных переменных или у возвращаемого значения

# OOP

In Python, object-oriented Programming (OOPs) is a programming paradigm that uses objects and classes in programming. It aims to implement real-world entities like inheritance, polymorphisms, encapsulation, etc. in the programming. The main concept of OOPs is to bind the data and the functions that work on that together as a single unit so that no other part of the code can access this data.

Язык может быть ориентирован на объекты, но программу, построенную по принципам ООП, можно написать и в языках, где нет как таковых классов/объектов. Например, на C, или на Go. 

- **Абстракция**. 

Выделение значимой информации и использование наиболее важных характеристик объекта.
Абстракция от общего к частному. Класс -> экземпляр класса, Базовый класс -> производный класс.

Выделяем общее из частного. Притом то общее, с которым мы хотим работать.

Пример абстракции. Если нам нужно, чтобы утка крякала и плавала, то все то, что крякает и плавает для нас будет уткой. Мы отбросили все частное, которое нам не нужно.

Также абстракция бывает горизонтальной. Когда мы моделируем нечто с помощью массива (множество, дерево и пр.). С помощью одного и того же инструмента можем определять разное.

Как это относится к ООП?


- **Инкапсуляция**

Логическое объединение данных и функций для работы с ними. Инструмент, который позволяет разделить действия, необходмые для разного уровня абстракций. 

Минимизация необходимого информационного пространства, за счет создания иерархии пространств имен (размещаем в объектке и его описание, и методы, которые с этим объектом работают). Нотация доступа к методам объекта через точку. Методы, поля и прочее и прочее и прочее. Посмотреть методы можно с помощью команды `dir()`.

Следует заметить, что также часто используется также "сокрытие" определенных полей. Когда мы говорим, что внутри объекта есть еще поля, но мы вам их не покажем. В некоторых языках есть, например, 


- **Наследование**

Повторное использование свойств объектов с описанием различий.

Хотим, когда мы используем очередной класс объектов, мы могли сказать "А можно сделать вот так же, как тут, только в некоторых пунктах мы делаем изменений". Повторное использование кода.


- **Полиморфизм**

Предоставление одинаковых средств взаимодействия с объектами разной природы. Возможность для одного и того же кода обрабатывать данные разных типов. Duck typing в Python -- это из коробки


Класс можно использовать как контейнер. Мы можем складывать в него дополнительные поля. Можно делать свои "пространства имен", и в них складывать какие-то методы. Например, можно сделать некоторый класс и объявить в нем свою функцию минимума. Или определить какое-то поле, например, value. И тогда value будет определена только в области видимости нашего класса

**Class** 
A class is a collection of objects. A class contains the blueprints or the prototype from which the objects are being created. It is a logical entity that contains some attributes and methods. 

To understand the need for creating a class let’s consider an example, let’s say you wanted to track the number of dogs that may have different attributes like breed, age. If a list is used, the first element could be the dog’s breed while the second element could represent its age. Let’s suppose there are 100 different dogs, then how would you know which element is supposed to be which? What if you wanted to add other properties to these dogs? This lacks organization and it’s the exact need for classes. 

Some points on Python class:  

Classes are created by keyword class.
Attributes are the variables that belong to a class.
Attributes are always public and can be accessed using the dot (.) operator. Eg.: Myclass.Myattribute

**Objects**
The object is an entity that has a state and behavior associated with it. It may be any real-world object like a mouse, keyboard, chair, table, pen, etc. Integers, strings, floating-point numbers, even arrays, and dictionaries, are all objects. More specifically, any single integer or any single string is an object. The number 12 is an object, the string “Hello, world” is an object, a list is an object that can hold other objects, and so on. You’ve been using objects all along and may not even realize it.

*An object consists of :*

- State: It is represented by the attributes of an object. It also reflects the properties of an object.

- Behavior: It is represented by the methods of an object. It also reflects the response of an object to other objects.

- Identity: It gives a unique name to an object and enables one object to interact with other objects.

## Обработка исключений

In [4]:
a = '1234'

try:
    a[0] = '10'
except:
    print('impossible to modify object')

impossible to modify object


In [5]:
a = '1234'

try:
    a[0] = '10'
except BaseException:
    print('impossible to modify object')

impossible to modify object


In [6]:
a = '1234'

try:
    a[0] = '10'
except BaseException:
    print('impossible to modify object')
finally:
    print('run finally in any case')

impossible to modify object
run finally in any case


In [7]:
a = [1, 2]

try:
    a[0] = '10'
except BaseException:
    print('impossible to modify object')
finally:
    print('run finally in any case')

run finally in any case


In [8]:
issubclass(TypeError, BaseException), issubclass(ValueError, BaseException)

(True, True)

In [10]:
TypeError.mro(), ValueError.mro()  # смотрим цепочку наследования

([TypeError, Exception, BaseException, object],
 [ValueError, Exception, BaseException, object])

In [11]:
class CustomStringException(BaseException):
    pass

In [18]:
a = '1234'

try:
    a[0] = '10'
    
except TypeError as e:
    raise CustomStringException(e)

else:
    print('successfully modified', a)

CustomStringException: 'str' object does not support item assignment

In [19]:
a = ['1234', 2, 3]

try:
    a[0] = '10'
    
except TypeError as e:
    raise CustomStringException(e)

else:
    print(f'successfully modified {a=}')

successfully modified a=['10', 2, 3]


In [20]:
raise CustomStringException('some message')

CustomStringException: some message

In [23]:
class A(Exception):
    pass

class B(A):
    pass

class C(B):
    pass

for cls in [C, A, B]:
    try:
        print(f'raised {cls.__name__}')
        raise cls()
    
    except A:
        print('A')
    
    except C:
        print('C')
        
    except B:
        print('B')
        
  

raised C
A
raised A
A
raised B
A


In [24]:
for cls in [C, A, B]:
    try:
        print(f'raised {cls.__name__}')
        raise cls()
    
    except B:
        print('B')
    
    except C:
        print('C')
        
    except A:
        print('A')
        
  

raised C
B
raised A
A
raised B
B


In [26]:
for cls in [C, A, B]:
    try:
        print(f'raised {cls.__name__}')
        raise cls()
    
    except C:
        print('C')
    
    except B:
        print('B')
        
    except A:
        print('A')
        
  

raised C
C
raised A
A
raised B
B


Родительское исключение будет перехватывать исключения всех своих наследников

## class as a decorator

Экземпляры класса могут быть callable. А если функция может быть декоратором, то что мешает тогда классу? Должен быть определен magic метод `__call__`

In [54]:
class ClassDecorator:
    def __init__(self, func):
        self.entry_str = '[decorating inner func]'
        self.function = func
        self.close_str = '[letting the song flow]'

     
    def __call__(self):
        print(self.entry_str)
        self.function()
        print(self.close_str)

 
 
@ClassDecorator
def function():
    print('Vanderlyle crybaby cry...')

In [56]:
function()

[decorating inner func]
Vanderlyle crybaby cry...
[letting the song flow]


Как и в декораторах-функциях, мы тоже можем возвращать значения (а не только использовать print) и делать параметры

Сделаем таймер

In [60]:
import time

class Timer:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args):
        t_start = time.time()
        res = self.func(*args)
        total_time = time.time() - t_start
        print(f'{self.func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
        return res

In [61]:
@Timer
def slow_func(seconds_to_sleep):
    time.sleep(seconds_to_sleep)
    return 73

In [62]:
slow_func(3)

slow_func((3,)) -> 73 executed in 3.00s


73

## class as decorated

Класс может быть тоже декорирован функцией или классом

In [27]:
def decorator_function(target):

    def decorator_init(self, *args):
        print("DECORATED")
    
    print('I changed __init__ in class')
    target.__init__ = decorator_init

    return target


class Target:

    def __init__(self, *args):
        self.arg_list = args
        print("Target running")


t = Target('one', 1)
t.__dict__

Target running


{'arg_list': ('one', 1)}

In [28]:
@decorator_function
class Target:

    def __init__(self, *args):
        self.arg_list = args
        print("Target running")

I changed __init__ in class


In [29]:
dt = Target('one', 1)
dt.__dict__

DECORATED


{}